# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fortuna,40.5982,-124.1573,12.19,76,40,5.66,US,1707081858
1,1,pokrovsk,48.2820,37.1758,2.79,90,100,5.42,UA,1707081859
2,2,salinas,36.6777,-121.6555,17.29,83,75,13.38,US,1707081859
3,3,avarua,-21.2078,-159.7750,29.03,74,20,2.57,CK,1707081859
4,4,sovetskiy,61.3614,63.5842,-17.28,92,97,2.24,RU,1707081860


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City")

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [14]:
#!pip install cartopy geoviews pyproj

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temperature_df = city_data_df.loc[(city_data_df["Max Temp"] >= 25)]
ideal_weather_df = ideal_temperature_df.loc[city_data_df["Cloudiness"] <= 40]

# Drop any rows with null values
clean_weather = ideal_weather_df.dropna()

# Display sample data
clean_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,avarua,-21.2078,-159.7750,29.03,74,20,2.57,CK,1707081859
20,20,alice springs,-23.7000,133.8833,28.75,32,37,6.17,AU,1707081582
48,48,osorno,-40.5667,-73.1500,31.81,16,0,4.12,CL,1707081867
54,54,acajutla,13.5928,-89.8275,29.27,65,35,6.29,SV,1707081868
65,65,isangel,-19.5500,169.2667,28.06,94,40,4.12,VU,1707081643
...,...,...,...,...,...,...,...,...,...,...
531,531,pinotepa nacional,16.3167,-98.0167,31.05,32,1,5.84,MX,1707081928
533,533,sao jose da coroa grande,-8.8978,-35.1478,27.40,76,38,3.39,BR,1707081929
543,543,sao vicente,-23.9631,-46.3919,26.10,56,0,1.54,BR,1707081634
570,570,les anglais,18.3054,-74.2197,26.70,71,19,2.62,HT,1707081933


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_weather[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = " "

# Display sample data
hotel_df

/var/folders/5k/_yxk9n810sqfyd87_kzfpfbm0000gn/T/ipykernel_34701/1843736880.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = " "


,City,Country,Lat,Lng,Humidity,Hotel Name
3,avarua,CK,-21.2078,-159.7750,74,
20,alice springs,AU,-23.7000,133.8833,32,
48,osorno,CL,-40.5667,-73.1500,16,
54,acajutla,SV,13.5928,-89.8275,65,
65,isangel,VU,-19.5500,169.2667,94,
...,...,...,...,...,...,...
531,pinotepa nacional,MX,16.3167,-98.0167,32,
533,sao jose da coroa grande,BR,-8.8978,-35.1478,76,
543,sao vicente,BR,-23.9631,-46.3919,56,
570,les anglais,HT,18.3054,-74.2197,71,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
   "apiKey": geoapify_key
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places/"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        #If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
avarua - nearest hotel: Paradise Inn
alice springs - nearest hotel: Aurora Alice Springs
osorno - nearest hotel: Hotel Sonesta Osorno
acajutla - nearest hotel: Hotel Lara
isangel - nearest hotel: Tanna Lodge
conde - nearest hotel: Paladino
kolda - nearest hotel: le firdou
santa cruz - nearest hotel: Aviador Hotel Boutique
nova vicosa - nearest hotel: Pousada Pontal da Barra
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
bathsheba - nearest hotel: Atlantis Hotel
kralendijk - nearest hotel: Hotel Bonaire Inn
ixtapa - nearest hotel: Ma. Cristina
acapulco de juarez - nearest hotel: Hotel del Valle
coahuayana de hidalgo - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
satuek - nearest hotel: No hotel found
saipan - nearest hotel: Chalan Kanoa Beach Hotel
puerto lempira - nearest hotel: Hotel Yu Baiwan
kingston - nearest hotel: Aphrodite Hotel
georgetown - nearest hotel: Page 63 hostel
freetown - nearest hotel: Formal Leona hotel
san antonio de

,City,Country,Lat,Lng,Humidity,Hotel Name
3,avarua,CK,-21.2078,-159.7750,74,Paradise Inn
20,alice springs,AU,-23.7000,133.8833,32,Aurora Alice Springs
48,osorno,CL,-40.5667,-73.1500,16,Hotel Sonesta Osorno
54,acajutla,SV,13.5928,-89.8275,65,Hotel Lara
65,isangel,VU,-19.5500,169.2667,94,Tanna Lodge
...,...,...,...,...,...,...
531,pinotepa nacional,MX,16.3167,-98.0167,32,No hotel found
533,sao jose da coroa grande,BR,-8.8978,-35.1478,76,Hotel do Sol
543,sao vicente,BR,-23.9631,-46.3919,56,Pousada Vitória
570,les anglais,HT,18.3054,-74.2197,71,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)